In [ ]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [ ]:
!pip install langchain-groq langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.3/469.3 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.2 which is incompatible.


In [ ]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage

In [ ]:
llm = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [ ]:
!pip install langchain_tavily langchain_community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.3 MB/s eta 0:00:00


In [ ]:
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Tavily API key: ··········


In [ ]:
from typing import List
from typing_extensions import TypedDict
from langchain_core.documents import Document

# Define the state structure
class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: The question to search for.
        documents: A list of scraped documents.
        sender: The name of the node that last modified the state.
    """
    question: str
    documents: List[Document]
    sender: str

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_tavily import TavilySearch
from langchain_core.documents import Document
from typing import List
from typing_extensions import TypedDict

# Define the state
class GraphState(TypedDict):
    question: str
    documents: List[Document]
    sender: str

# --- Your Function (with st.warning) ---
def web_search_node(state: GraphState) -> GraphState:
    """Searches the web for information, then scrapes the content from the resulting URLs."""
    print("--- Calling Web Search Node ---")
    question = state["question"]

    tavily_search = TavilySearch(
        max_results=2,
        search_depth="advanced",
        include_domains=["who.int"]
    )
    search_results = tavily_search.invoke(question)

    scraped_docs = []
    if not search_results or "results" not in search_results:
        print("No search results found.")
        return {"documents": [], "sender": "web_search_node"}

    urls = [result.get("url") for result in search_results["results"] if result.get("url")]
    print(f"Found URLs: {urls}")

    if not urls:
        print("No valid URLs in search results.")
        return {"documents": [], "sender": "web_search_node"}

    for url in urls:
        try:
            print(f"Scraping {url}...")
            loader = WebBaseLoader(url)
            docs = loader.load()
            for doc in docs:
                doc.metadata["source"] = url
            scraped_docs.extend(docs)
        except Exception as e:
            print(f"Error scraping {url}: {e}")

    print(f"Scraped {len(scraped_docs)} documents.")
    return {"documents": scraped_docs, "sender": "web_search_node"}

In [ ]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(GraphState)


workflow.add_node("web_search", web_search_node)

workflow.set_entry_point("web_search")

workflow.add_edge("web_search", END)

app = workflow.compile()


inputs = {"question": "What is the latest WHO report on malaria?"}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"\n--- Output from Node: {key} ---")

        if 'documents' in value and value['documents']:
            print(f"Documents found: {len(value['documents'])}")
            print("-" * 30)

            for i, doc in enumerate(value['documents']):
                print(f"\n--- Document {i+1} ---")
                print(f"Source: {doc.metadata.get('source', 'No source available')}")
                # print("\nContent:")
                # print(doc.page_content)
        else:
            print("No documents found in this step.")

--- Calling Web Search Node ---
Found URLs: ['https://www.who.int/teams/global-malaria-programme/reports/world-malaria-report-2024', 'https://www.who.int/campaigns/world-malaria-day/2025']
Scraping https://www.who.int/teams/global-malaria-programme/reports/world-malaria-report-2024...
Scraping https://www.who.int/campaigns/world-malaria-day/2025...
Scraped 2 documents.

--- Output from Node: web_search ---
Documents found: 2
------------------------------

--- Document 1 ---
Source: https://www.who.int/teams/global-malaria-programme/reports/world-malaria-report-2024

--- Document 2 ---
Source: https://www.who.int/campaigns/world-malaria-day/2025


In [ ]:
tavily_search = TavilySearch(
      max_results=2,
      search_depth="advanced",
      include_domains=["who.int"],
      include_raw_content=True
  )

In [ ]:
question = "What is the latest WHO report on malaria?"
search_results = tavily_search.invoke(question)